In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import NearestNeighbors

## 1. Import des 2 DS : title_basic et title_ratings

In [2]:
# title.basics
title_basics = pd.read_csv('https://datasets.imdbws.com/title.basics.tsv.gz', sep ='\t', low_memory = False)

In [3]:
# title.ratings
title_ratings= pd.read_csv('https://datasets.imdbws.com/title.ratings.tsv.gz', sep = '\t')

In [4]:
print("Title basics: ", title_basics.shape)
print("Title ratings: ", title_ratings.shape)

Title basics:  (8479240, 9)
Title ratings:  (1203319, 3)


In [5]:
# faire une merge du df_genre avec ratings : inner join car je ne veux que les données qui sont communes aux 2 DS
df_rating = title_basics.merge(title_ratings, on='tconst', how='inner')
df_rating.reset_index(inplace = True, drop= True)

In [6]:
print("Shape après merge: ", df_rating.shape)

Shape après merge:  (1203319, 11)


## 2. Appliquer les filtres :
- ne garder que les films
- ne garder que les films où isadult == 0
- ne garder que les films ayant une note de plus de la moyenne des notes
- ne garder que les films ayant un nombre de vote supérieur à 500


Après gestion des Nan :
- films de plus de 60min et moins de 240

In [7]:
# ne garder que les films : 270K de films
movies_df = df_rating[df_rating['titleType'] == 'movie']
movies_df.shape

(270788, 11)

In [8]:
# supprimer colonne endYear qui ne concerne visiblement que les séries
movies_df.drop('endYear', axis = 1, inplace = True)

C:\Users\Alex\anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


# IsAdult == 0 : 265K films



In [9]:
movies_df = movies_df[movies_df['isAdult'] == '0']
movies_df.drop('isAdult', axis = 1, inplace = True)
movies_df.shape

(266379, 9)

In [10]:
# ne garder que les films dont le rating est supérieur à la moy de tous les films
movies_df = movies_df[movies_df['averageRating'] > movies_df['averageRating'].mean()]
movies_df.shape

(144825, 9)

In [11]:
movies_df['numVotes'].describe()


count    1.448250e+05
mean     5.311516e+03
std      4.432391e+04
min      5.000000e+00
25%      1.600000e+01
50%      5.000000e+01
75%      3.180000e+02
max      2.488195e+06
Name: numVotes, dtype: float64

In [12]:
# ne garder que les films qui ont plus de 500 votes
movies_df = movies_df[movies_df['numVotes'] > 500]
movies_df.shape

(30385, 9)

## 3. Valeurs manquantes

In [13]:
movies_df = movies_df.replace('\\N', np.NaN)

In [14]:
movies_df.isna().sum()

tconst             0
titleType          0
primaryTitle       0
originalTitle      0
startYear          1
runtimeMinutes    62
genres             2
averageRating      0
numVotes           0
dtype: int64

In [15]:
# suppression des lignes où il y a des valeurs manquantes
movies_df.dropna(inplace = True)

In [16]:
movies_df.shape

(30322, 9)

## 4. Types de données

In [17]:
movies_df.dtypes

tconst             object
titleType          object
primaryTitle       object
originalTitle      object
startYear          object
runtimeMinutes     object
genres             object
averageRating     float64
numVotes            int64
dtype: object

In [18]:
movies_df.head()

,tconst,titleType,primaryTitle,originalTitle,startYear,runtimeMinutes,genres,averageRating,numVotes
1002,tt0002130,movie,Dante's Inferno,L'Inferno,1911,71,"Adventure,Drama,Fantasy",7.0,2706
1098,tt0002423,movie,Passion,Madame DuBarry,1919,85,"Biography,Drama,Romance",6.7,864
1196,tt0002844,movie,Fantômas: In the Shadow of the Guillotine,Fantômas - À l'ombre de la guillotine,1913,54,"Crime,Drama",7.0,2181
1226,tt0003014,movie,Ingeborg Holm,Ingeborg Holm,1913,96,Drama,7.0,1161
1231,tt0003037,movie,Fantomas: The Man in Black,Juve contre Fantômas,1913,61,"Crime,Drama",7.0,1510


In [19]:
movies_df['runtimeMinutes'] = movies_df['runtimeMinutes'].astype(int)
movies_df['startYear'] = movies_df['startYear'].astype(int)

In [20]:
# ne garder que les films dont la durée est supérieure à 60min et inférieur à 240min
movies_df = movies_df[(movies_df['runtimeMinutes'] > 60) & (movies_df['runtimeMinutes'] < 240)]

In [21]:
# transformer la colonne genre avec get_dummies
movies_df = pd.concat([movies_df, movies_df['genres'].str.get_dummies(sep = ',')], axis = 1)

In [33]:
movies_df.iloc[:, 7:]

,averageRating,numVotes,Action,Adult,Adventure,Animation,Biography,Comedy,Crime,Documentary,...,Music,Musical,Mystery,News,Romance,Sci-Fi,Sport,Thriller,War,Western
0,7.0,2706,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,6.7,864,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,7.0,1161,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,7.0,1510,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,7.0,1180,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29933,7.2,563,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29934,8.2,671,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29935,6.8,706,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
29936,7.6,3219,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
movies_df.reset_index(inplace=True, drop = True)

## 5. Algorithme 1

In [34]:
# définition des colonnes de features
colonnes = list(movies_df.columns)
col = colonnes[7:]
X = movies_df[col]

# entrainement du modèle
model = NearestNeighbors(n_neighbors=6).fit(X)

# assigner les distances et les indices obtenus 
distances, indices = model.kneighbors(X)

## 6. Test 1 : non concluant 

Lorsque l'utilisateur va entrer le nom d'un film qu'il a aimé : il faut que l'app puisse la retrouver dans la liste des films dispo dans le dataset

In [35]:
# retourne l'indice du film quand on lui donne le nom complet
def get_index(title):
    return movies_df[movies_df['primaryTitle'] == title].index.tolist()[0]

In [36]:
# si l'utilisateur ne tape pas le nom en entier
# on recherche toutes les entrées où il y a ce nom et on prend le premier
def get_index_from_partial_title(title):
    title = title.capitalize()
    return movies_df[movies_df['primaryTitle'].str.contains(title)].index.tolist()[0]

In [37]:
# display les titres proches 
def get_movies(title):
    movie_id = get_index(title)
    # pour tous les indices de l'array indices crée dans le modèle, trouve les voisins du titre donné
    # le 0 étant le film lui même (distance 0)
    for id in indices[movie_id][1:]:
        print(movies_df.iloc[id, 2])
    

In [38]:
get_movies('Star Wars: Episode IV - A New Hope')

The Wolf of Wall Street
Schindler's List
Saving Private Ryan
The Prestige
The Avengers


## 7. Algo 2 et test

Changer la métrique dans l'algo en 'cosine' qui ne calcule plus la distance euclidienne entre deux titres mais le cosinus entre les deux vecteurs. 

In [39]:
# retourne l'indice du film quand on lui donne le nom complet
def get_index(title):
    return movies_df[movies_df['primaryTitle'] == title].index.tolist()[0]

# si l'utilisateur ne tape pas le nom en entier
# on recherche toutes les entrées où il y a ce nom et on prend le premier
def get_index_from_partial_title(title):
    title = title.capitalize()
    return movies_df[movies_df['primaryTitle'].str.contains(title)].index.tolist()[0]

In [40]:
# metric = cosine
# entrainement du modèle
model2 = NearestNeighbors(n_neighbors=6, metric='cosine').fit(X)

# assigner les distances et les indices obtenus 
distances, indices = model2.kneighbors(X)

In [41]:
# display les titres proches 
def get_movies(title):
    movie_id = get_index(title)
    # pour tous les indices de l'array indices crée dans le modèle, trouve les voisins du titre donné
    # le 0 étant le film lui même (distance 0)
    for id in indices[movie_id][1:]:
        print(movies_df.iloc[id, 2])

In [42]:
get_movies('Star Wars: Episode IV - A New Hope')

Avatar
Star Wars: Episode V - The Empire Strikes Back
Pirates of the Caribbean: The Curse of the Black Pearl
Batman Begins
Gladiator


In [43]:
get_movies('Batman Begins')

Gladiator
The Lord of the Rings: The Two Towers
The Avengers
The Lord of the Rings: The Return of the King
Star Wars: Episode IV - A New Hope


In [44]:
get_movies("Bridget Jones's Diary")

Vicky Cristina Barcelona
Jerry Maguire
Don Jon
Marriage Story
The Lobster


## 8. Input user : pour streamlit

In [45]:
# copie du df pour modifier :
movies_lower = movies_df.copy()
movies_lower['primaryTitle'] = movies_lower['primaryTitle'].str.lower()

In [46]:
# sortir une liste de film en fonction de ce qu'il entre:
def get_list_from_partial_title(title):
    title = title.lower()
    list_movies= movies_lower[movies_lower['primaryTitle'].str.contains(title)].index.tolist()
    df = movies_df.iloc[list_movies, :]
    titres = []
    for key, value in df.iteritems() :
        if key == 'primaryTitle':
            titres.append(value)
    return titres

In [47]:
get_list_from_partial_title('Star wars')

[7087                 Star Wars: Episode IV - A New Hope
 7504     Star Wars: Episode V - The Empire Strikes Back
 8142         Star Wars: Episode VI - Return of the Jedi
 11592         Star Wars: Episode I - The Phantom Menace
 11601      Star Wars: Episode II - Attack of the Clones
 11602      Star Wars: Episode III - Revenge of the Sith
 23713        Star Wars: Episode VII - The Force Awakens
 23763           Star Wars: Episode VIII - The Last Jedi
 23764     Star Wars: Episode IX - The Rise of Skywalker
 25104       Plastic Galaxy: The Story of Star Wars Toys
 25240                      Rogue One: A Star Wars Story
 25282                           Solo: A Star Wars Story
 Name: primaryTitle, dtype: object]

In [48]:
movies_df.to_csv('df_algo.csv')